# Library importing

In [3]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import joblib

# data pre-proccessing

## data reading

In [4]:
final_data = pd.read_csv("/kaggle/input/depi-dataset/preprocessing")
final_data.drop(columns='Unnamed: 0',inplace =True)

In [5]:
preprocessor=joblib.load("/kaggle/input/pre-processing-pipeline/other/default/1/preprocessing_pipeline.pkl")

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OrdinalEncoder from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info 

## Data spliting

In [6]:
y=final_data['sales']
x=final_data.drop('sales',axis = 1)

# Model

In [7]:
x=preprocessor.fit_transform(x)

In [8]:
model = XGBRegressor(
    objective = 'reg:squarederror',
    device='cuda',
    booster =  "gbtree",
    max_depth = 10,
    random_state = 42,
    learning_rate = 0.05,
    eval_metric = 'mae',
    early_stopping_rounds = 40,
    reg_lambda = 2,
    tree_method= 'gpu_hist',
    n_estimators=2000,
    subsample=0.5
)

In [9]:
x_temp , x_test , y_temp , y_test = train_test_split(x , y , test_size = 0.15 , random_state = 42)
x_train , x_val , y_train , y_val = train_test_split(x_temp , y_temp , test_size = 0.15 , random_state = 42)

In [10]:
model.fit(x_train,y_train,eval_set = [(x_val , y_val)])

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [19:29:17] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[0]	validation_0-mae:1.51432
[1]	validation_0-mae:1.46063
[2]	validation_0-mae:1.41046
[3]	validation_0-mae:1.36423
[4]	validation_0-mae:1.32126
[5]	validation_0-mae:1.28117
[6]	validation_0-mae:1.24419
[7]	validation_0-mae:1.21019
[8]	validation_0-mae:1.17826
[9]	validation_0-mae:1.14881
[10]	validation_0-mae:1.12161
[11]	validation_0-mae:1.09636
[12]	validation_0-mae:1.07248
[13]	validation_0-mae:1.05016
[14]	validation_0-mae:1.02970
[15]	validation_0-mae:1.01097
[16]	validation_0-mae:0.99323
[17]	validation_0-mae:0.97603
[18]	validation_0-mae:0.96038
[19]	validation_0-mae:0.94547
[20]	validation_0-mae:0.93166
[21]	validation_0-mae:0.91888
[22]	validation_0-mae:0.90640
[23]	validation_0-mae:0.89454
[24]	validation_0-mae:0.88416
[25]	validation_0-mae:0.87347
[26]	validation_0-mae:0.86320
[27]	validation_0-mae:0.85413
[28]	validation_0-mae:0.84451
[29]	validation_0-mae:0.83637
[30]	validation_0-mae:0.82902
[31]	validation_0-mae:0.82072
[32]	validation_0-mae:0.81382
[33]	validation_0-ma

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cuda', early_stopping_rounds=40,
             enable_categorical=False, eval_metric='mae', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=2000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [13]:
model.score(x_train,y_train) 

0.7266027211225974

In [14]:
model.score(x_test,y_test)

0.6804995201741726